In [1]:
import pandas as pd
import numpy as np
import ruptures as rp
from prediction_pipeline.preprocessing import *
from sklearn.linear_model import LinearRegression
from glob import glob
%load_ext autoreload
%autoreload 2

from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.cross_decomposition import PLSRegression
from collections import defaultdict 
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import RobustScaler
import os
import subprocess

/home/hv270/platelets/lib/python3.7/site-packages/FlowCytometryTools/core/docstring.py:56: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  doc = dedent(doc)


In [2]:
data_dir = "/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/data_home"
export_folder = "/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing"

In [3]:
tag = "20_dec_21"
tag_path = "%s/%s" % (export_folder,tag) 
tag_score_path = "%s/Genetic_score/%s" % (export_folder,tag) 
if not os.path.exists(tag_path): os.mkdir(tag_path)
if not os.path.exists(tag_score_path): os.mkdir(tag_score_path)

# Tags

## 9jun

- factor analysis pour décomposer les scattergrams.
- PLS pour l'ajustement des features et l'estimation par les covariates
- PCT enlevé des covariates, IPF y est bien par contre
- N_PLS_COMPS = 1

## 10jun

idem + :
- phase A & B, 2K cellules / scattergram

## 11jun

idem + :
- PCT gardé comme covariate
- PCA pour les features au lieu de FA
- ajustement des features par PLS mais avec 3 composantes. PLS fittée séparément pour int et sys

jusqu'ici, PEAR1 avait une P-value autour de 0,1... mais avec ça, elle passe à 1E-7. C'est bien, mais il faut pousser encore. Je pense que c'est l'ajustement qui compte beaucoup.

## 11jun_WE

idem + :
- 4 composantes dans la PLS d'ajustement
- On essaie le GWAS sur les _FIB

-> On a PEAR1 ! :) 

## 15jun

idem + :
- normalisé chaque features par sa STD après ajustement, séparément pour train et pour interval

-> Ca marche moins bien...

## 16jun

- enlevé la normalisation
- Remarqué que la covariance est différente entre les variables ajustées de INT et de TRAIN, donc X_int = PCA_train_inv(PCA_int(X_int))

-> Ne marche pas

## 21 juin

- même process, en sélectionnant moins de features (6)

-> PEAR1 est à 4e-7 en PLINK. À suivre

## 21 juin no_pca

- même process, enlevé la PCA

-> 4e-7 mais Beta opposé !?

## 21jun_nopca_3feats

- on ne garde que 3 features

-> 6e-7

## 21jun_3feats_fixedpca

- Problème de PCA résolu : on s'assure que les composantes des PCA post-ajustement sont à peu près les mêmes (on ne vérifiait pas que les signes concordaient...)

-> Marche pas mieux, la PCA n'est pas adaptée

## 22jun_cholesky

- On décorrèle et recorrèle les données avec une Cholesky
- On repasse avec 6 features
- On clippe à -4 - +4
- On remultiplie par les STD post-ajustement -> 1e-7 !

## 22jun_cholesky_3feats

- idem avec 3 features
- Vu qu'il y a peu de features, on utilise juste une régression linéaire à la place de la PLS pour la prédiction des phenotypes. Ca marche mieux (on explique plus de R^2) -> 8e-7

## 22jun_cholesky_3feats_adjust

- en plus, on fait un ajustement à la toute fin du processus

-> 7e-7

## 22jun_3feats_adjust

- No cholesky
- min cohort size = 200

-> 1.7e-7

## 22jun_adjust

- Back to 6 features

-> 2e-8 !!

## 22jun_adj_feats

- Adjust features before fit and prediction
- PLS instead of LR
- back to all features

-> 4.7e-9 !

- Try adding cholesky adjustment, it breaks the p-value. Maybe there's a better way to 

## 25jun_sep_PCA

- Try to use different PCAs for both cohorts

-> 2e-8

## 5_july_gating

- Changed gating
- Kept more platelets per sample
- Kept individuals with PLT count discrepancies
- Same PCA for both cohorts

## 26 july

- Filter upper left corner (kept first rows of .fcs and .116 files)
- adjust STD of scatter signal -> marche pas

## 27 july

-> Ce qui marche le mieux est l'ajustement à la toute fin. Voir si prendre les données de scatter ajustées aide.
-> Dernier truc à explorer : time_of_the_day x machine & regarder les cohortes qui n'ont pas un bon p

## 25 oct

- PCA par individu pour trouver les cohortes, puis par cohorte

In [4]:
features_cols = pd.read_hdf("%s/processed/features.hdf" % data_dir).columns
unadjusted_features_cols = ["%s_unadjusted" % f for f in features_cols]

# Matching tables

In [5]:
matching = pd.read_csv("/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_phenotypes/INTERVAL_OmicsMap_20200514.csv", sep=",")
has_one_id = ~matching["Affymetrix_QC_bl"].isnull() | ~matching["Affymetrix_QC_24m"].isnull()
matching = matching.loc[has_one_id]
matching["AffyID"] = matching["Affymetrix_QC_bl"]
matching.loc[matching.AffyID.isnull(),"AffyID"] = matching.loc[matching.AffyID.isnull(),"Affymetrix_QC_24m"]

In [6]:
int_phen, meta = get_INTERVAL_phenotypes()

Loading INTERVAL phenotypes
(48460, 760)
(45040, 760)
(38832, 760)
(112535, 11)
(102595, 11)
(17768, 772)
(32574, 777)
(32727, 777)
Checked that each Sample has one row max
Removing samples with less than 10 measurements per day
(30725, 782)


In [7]:
meta = pd.merge(matching,int_phen,left_on="identifier", right_on="identifier")
meta["IID"] = meta["AffyID"].astype(int).astype(str)
meta["FID"] = meta["AffyID"].astype(int).astype(str)
meta = meta.loc[meta.ethnicPulse == "Eng/W/Scot/NI/Brit"]
meta["cohort"] = meta.interval

In [8]:
meta.cohort.value_counts()

M12    5025
M10    4956
F14    4948
M08    4877
F12    4868
F16    4856
Name: cohort, dtype: int64

## Load table from 5B

In [9]:
X = pd.read_csv("%s/processed/predictions_lassoCV.csv" % data_dir,index_col="SampleNo")

In [10]:
X = pd.merge(X,meta[["IID","FID","SampleNo","cohort"]].rename(columns={"cohort":"interval_cohort"}),left_index=True,right_on="SampleNo",how="inner")
del X["SampleNo"]

In [11]:
X.columns

Index(['ADP_pred', 'SEX', 'AGE', 'PLT', 'MPV', 'PDW', 'PCT', 'IPF', 'cohort',
       'median_0', 'median_1', 'upper_0', 'upper_1', 'lower_0', 'lower_1',
       'std_0', 'std_1', 'iqr_0', 'iqr_1', 'ratio_upper', 'ratio_lower',
       'ratio_std', 'std_2_high', 'std_2_low', 'machine', 'cohort_all',
       'ADP_PSEL', 'ADP_FIB', 'ADP', 'CRP_PSEL', 'CRP_FIB', 'CRP', 'TRAP_PSEL',
       'TRAP_FIB', 'TRAP', 'PAR1_PSEL', 'PAR1_FIB', 'PAR1', 'PAR4_PSEL',
       'PAR4_FIB', 'PAR4', 'PEAR1', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6',
       'PC7', 'PC8', 'PC9', 'PC10', 'Instrument', 'days_since_start', 'IID',
       'FID', 'interval_cohort'],
      dtype='object')

In [12]:
covar_columns = ["SEX","AGE","interval_cohort"] + ["PC%d" % d for d in range(1,10)]

# GWAS just on PEAR1

## With PCs

In [43]:
Y = X[["FID","IID"] + ["ADP_pred"]]
C = X[["FID","IID"] + ["AGE","SEX"] + ["PC%d" % d for d in range(1,10)]]
cohort_dummies = pd.get_dummies(X["interval_cohort"],drop_first=True)
C[cohort_dummies.columns] = cohort_dummies

In [44]:
Y.to_csv("%s/%s/hippo_ADP_pred.csv" % (export_folder,tag), index=False,header=True, sep="\t")
C.to_csv("%s/%s/covariates_no_sysmex_PCs.csv" % (export_folder,tag), index=False,header=True, sep="\t")

In [45]:
k = 1
n_splits = 4
command = "/home/hv270/plink2"

command += " --bfile /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_%d" % k
command += " --glm hide-covar"
command += " --extract /home/hv270/pear1"
command += " --keep %s/king-cutoff.king.cutoff.in.id" % export_folder
command += " --pheno %s/%s/hippo_ADP_pred.csv" % (export_folder, tag)
command += " --covar %s/%s/covariates_no_sysmex_PCs.csv" % (export_folder, tag)
command += " --out %s/%s/chr_%d_ADP_PCs" % (export_folder, tag,k)
command += " --threads %d" % n_splits
command += " --allow-no-sex"
command += " --variance-standardize"

In [46]:
command

'/home/hv270/plink2 --bfile /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_1 --glm hide-covar --extract /home/hv270/pear1 --keep /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/king-cutoff.king.cutoff.in.id --pheno /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/20_dec_21/hippo_ADP_pred.csv --covar /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/20_dec_21/covariates_no_sysmex_PCs.csv --out /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/20_dec_21/chr_1_ADP_PCs --threads 4 --allow-no-sex --variance-standardize'

In [47]:
p = subprocess.Popen(command,shell=True)
p.wait()

0

In [48]:
files = glob("%s/%s/chr_1_ADP_PCs.*.glm.linear" % (export_folder,tag))

In [49]:
files

['/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/20_dec_21/chr_1_ADP_PCs.ADP_pred.glm.linear']

In [50]:
dfs = []
for f in sorted(files):
    df = pd.read_csv(f,sep="\t")
    df["pheno"] = f.split(".")[-3]
    dfs.append(df)
df = pd.concat(dfs,axis=0)

In [51]:
df

,#CHROM,POS,ID,REF,ALT,A1,TEST,OBS_CT,BETA,SE,T_STAT,P,pheno
0,1,156869047,rs12566888,T,G,T,ADD,23546,-0.159631,0.015727,-10.1502,3.707650e-24,ADP_pred


## Run without MPV (whole genome)

In [53]:
Y = X[["FID","IID"] + ["ADP_pred"]]
C = X[["FID","IID"] + ["AGE","SEX"] + ["PC%d" % d for d in range(1,10)]]
cohort_dummies = pd.get_dummies(X["interval_cohort"],drop_first=True)
C[cohort_dummies.columns] = cohort_dummies

Y.to_csv("%s/%s/hippo_ADP.csv" % (export_folder,tag), index=False,header=True, sep="\t")
C.to_csv("%s/%s/covariates_PCs_noMPV.csv" % (export_folder,tag), index=False,header=True, sep="\t")

In [54]:
n_splits = 12
for k in range(1,23):
    command = "/home/hv270/plink2 "

    command += "--bfile /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_%d " % k
    command += "--glm hide-covar"
    command += " --keep %s/king-cutoff.king.cutoff.in.id" % export_folder
    command += " --pheno %s/%s/hippo_ADP.csv" % (export_folder, tag)
    command += " --covar %s/%s/covariates_PCs_noMPV.csv" % (export_folder, tag)
    command += " --out %s/%s/chr_%d_ADP_noMPV" % (export_folder, tag,k)
    command += " --threads %d" % n_splits
    command += " --allow-no-sex"
    command += " --variance-standardize"
    
    f_name = "/home/hv270/interval_slurm_calls/call_chr_%d_sysmex.sh" % k
    f = open(f_name,"w+")
    f.write("#!/bin/sh\n")

    f.write("#SBATCH -J chr_%d\n" % k)
    f.write("#SBATCH -A ASTLE-SL2-CPU\n")
    f.write("#SBATCH -N 1\n")
    f.write("#SBATCH -n %d\n" % n_splits)
    #f.write("#SBATCH -c %d\n" % n_splits)
    f.write("#SBATCH --time=0:15:00\n")
    f.write("#SBATCH -p skylake-himem\n")
    f.write("#SBATCH --output=/home/hv270/slurm_log/%s_%s_ADP_no_MPV.out\n" % (tag,k))
    f.write("export OMP_NUM_THREADS=%d\n" % n_splits)

    f.write(". /etc/profile.d/modules.sh\n")
    f.write("module purge\n")
    f.write("module load rhel7/default-peta4\n")
    f.write(command)
    f.close()
    print(command)
    
    p = subprocess.Popen("sbatch %s" % f_name,shell=True)
    p.wait()

/home/hv270/plink2 --bfile /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_1 --glm hide-covar --keep /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/king-cutoff.king.cutoff.in.id --pheno /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/20_dec_21/hippo_ADP.csv --covar /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/20_dec_21/covariates_PCs_noMPV.csv --out /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/20_dec_21/chr_1_ADP_noMPV --threads 12 --allow-no-sex --variance-standardize
/home/hv270/plink2 --bfile /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_2 --glm hide-covar --keep /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/king-cutoff.king.cutoff.in.id --pheno /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_pro

/home/hv270/plink2 --bfile /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_15 --glm hide-covar --keep /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/king-cutoff.king.cutoff.in.id --pheno /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/20_dec_21/hippo_ADP.csv --covar /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/20_dec_21/covariates_PCs_noMPV.csv --out /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/20_dec_21/chr_15_ADP_noMPV --threads 12 --allow-no-sex --variance-standardize
/home/hv270/plink2 --bfile /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_16 --glm hide-covar --keep /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/king-cutoff.king.cutoff.in.id --pheno /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_

# Run with Bolt

In [13]:
pheno_cols = ["ADP_pred"]
n_splits = 32

In [14]:
Y = X[["FID","IID"] + ["ADP_pred"]]
C = X[["FID","IID"] + ["AGE","SEX","interval_cohort"] + ["PC%d" % d for d in range(1,10)]]

Y.to_csv("%s/%s/hippo_ADP.csv" % (export_folder,tag), index=False,header=True, sep="\t")
C.to_csv("%s/%s/covariates_PCs_noMPV_bolt.csv" % (export_folder,tag), index=False,header=True, sep="\t")

## All chromosomes together (hasn't time to run on imputed SNPs)

In [19]:
for pheno in pheno_cols:

    command = "/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/BOLT-LMM_v2.3.4/bolt "
    for k in range(1,23):
        command += "--bim=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_%d.bim " % k
    # Only one fam file has to be specified
    command += "--fam=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_1.fam "
    for k in range(1,23):
        command += "--bed=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_%d.bed " % k
    command += "--lmm \
        --LDscoresFile=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/BOLT-LMM_v2.3.4/tables/LDSCORE.1000G_EUR.tab.gz \
        --phenoFile=%s/%s/hippo_ADP.csv \
        --phenoCol=%s " % (export_folder,tag,pheno)
    command += "--covarFile=%s/%s/covariates_PCs_noMPV_bolt.csv " % (export_folder, tag)
    command += "--maxMissingPerSnp=0.1 \
        --qCovarCol=AGE \
        --qCovarCol=PC1 \
        --qCovarCol=PC2 \
        --qCovarCol=PC3 \
        --qCovarCol=PC4 \
        --qCovarCol=PC5 \
        --qCovarCol=PC6 \
        --qCovarCol=PC7 \
        --qCovarCol=PC8 \
        --qCovarCol=PC9 \
        --covarCol=SEX \
        --covarCol=interval_cohort \
        --bgenMinMAF=1e-2 \
        --bgenMinINFO=0.4 \
        --maxModelSnps=2000000 \
        --lmmForceNonInf \
        --verboseStats \
        --sampleFile=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/alexander_only/interval.sample "
    command += "--statsFile=%s/%s/bolt_res_feb_%s " % (export_folder,tag,pheno)
    command += "--statsFileBgenSnps=%s/%s/bolt_res_bgen_feb_%s " % (export_folder,tag,pheno)
    command += "--modelSnps=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/indep_snps \
        --bgenFile=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/alexander_only/impute_{1:22}_interval.bgen \
        --numThreads=%d" % n_splits

    f_name = "/home/hv270/interval_slurm_calls/call_%s.sh" % pheno
    f = open(f_name,"w+")
    f.write("#!/bin/sh\n")

    f.write("#SBATCH -J %s\n" % (pheno))
    f.write("#SBATCH -A ASTLE-SL2-CPU\n")
    f.write("#SBATCH -N 1\n")
    f.write("#SBATCH -n %d\n" % n_splits)
    #f.write("#SBATCH -c %d\n" % n_splits)
    f.write("#SBATCH --time=8:00:00\n")
    f.write("#SBATCH -p skylake-himem\n")
    f.write("#SBATCH --output=/home/hv270/slurm_log/%s_%s.out\n" % (tag,pheno))
    f.write("export OMP_NUM_THREADS=%d\n" % n_splits)

    f.write(". /etc/profile.d/modules.sh\n")
    f.write("module purge\n")
    f.write("module load rhel7/default-peta4\n")
    f.write(command)
    f.close()

    print("Submitting %s" % pheno)
    p = subprocess.Popen("sbatch %s" % f_name,shell=True,stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    p.wait()

Submitting ADP_pred


## Group chromosomes

In [38]:
k = 4 # number of chromosomes processed together
groups = np.random.choice(22,22,replace=False) + 1
groups = [groups[i:(i+4)] for i in range(22) if i % 4 == 0]
n_splits = 16

In [42]:
for group in groups:
    group_ID = "_".join(["%d" % i for i in group])
    group_ID = "chr_" + group_ID
    for pheno in pheno_cols:

        command = "/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/BOLT-LMM_v2.3.4/bolt "
        for k in range(1,23):
            command += "--bim=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_%d.bim " % k
        # Only one fam file has to be specified
        command += "--fam=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_1.fam "
        for k in range(1,23):
            command += "--bed=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_%d.bed " % k
        command += "--lmm \
            --LDscoresFile=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/BOLT-LMM_v2.3.4/tables/LDSCORE.1000G_EUR.tab.gz \
            --phenoFile=%s/%s/hippo_ADP.csv \
            --phenoCol=%s " % (export_folder,tag,pheno)
        command += "--covarFile=%s/%s/covariates_PCs_noMPV_bolt.csv " % (export_folder, tag)
        command += "--maxMissingPerSnp=0.1 \
            --qCovarCol=AGE \
            --qCovarCol=PC1 \
            --qCovarCol=PC2 \
            --qCovarCol=PC3 \
            --qCovarCol=PC4 \
            --qCovarCol=PC5 \
            --qCovarCol=PC6 \
            --qCovarCol=PC7 \
            --qCovarCol=PC8 \
            --qCovarCol=PC9 \
            --covarCol=SEX \
            --covarCol=interval_cohort \
            --bgenMinMAF=1e-2 \
            --bgenMinINFO=0.4 \
            --maxModelSnps=2000000 \
            --lmmForceNonInf \
            --verboseStats \
            --sampleFile=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/alexander_only/interval.sample "
        command += "--statsFile=%s/%s/bolt_res_feb_%s_%s " % (export_folder,tag,pheno,group_ID)
        command += "--statsFileBgenSnps=%s/%s/bolt_res_bgen_feb_%s_%s " % (export_folder,tag,pheno,group_ID)
        command += "--modelSnps=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/indep_snps "
        for k in group:
            command += "--bgenFile=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/alexander_only/impute_%d_interval.bgen " % k
        command += "--numThreads=%d" % n_splits

        f_name = "/home/hv270/interval_slurm_calls/call_%s_%s.sh" % (pheno,group_ID)
        f = open(f_name,"w+")
        f.write("#!/bin/sh\n")

        f.write("#SBATCH -J %s\n" % (pheno))
        f.write("#SBATCH -A ASTLE-SL2-CPU\n")
        f.write("#SBATCH -N 1\n")
        f.write("#SBATCH -n %d\n" % n_splits)
        #f.write("#SBATCH -c %d\n" % n_splits)
        f.write("#SBATCH --time=8:00:00\n")
        f.write("#SBATCH -p skylake-himem\n")
        f.write("#SBATCH --output=/home/hv270/slurm_log/%s_%s_%s.out\n" % (tag,pheno, group_ID))
        f.write("export OMP_NUM_THREADS=%d\n" % n_splits)

        f.write(". /etc/profile.d/modules.sh\n")
        f.write("module purge\n")
        f.write("module load rhel7/default-peta4\n")
        f.write(command)
        f.close()

        print("Submitting %s" % pheno)
        p = subprocess.Popen("sbatch %s" % f_name,shell=True,stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        p.wait()

Submitting ADP_pred
Submitting ADP_pred
Submitting ADP_pred
Submitting ADP_pred
Submitting ADP_pred
Submitting ADP_pred


In [40]:
command

'/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/BOLT-LMM_v2.3.4/bolt --bim=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_7.bim --bim=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_1.bim --fam=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_1.fam --bed=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_7.bed --bed=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_1.bed --lmm             --LDscoresFile=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/BOLT-LMM_v2.3.4/tables/LDSCORE.1000G_EUR.tab.gz             --phenoFile=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/20_dec_21/hippo_ADP.csv             --phenoCol=ADP_pred --covarFile=/home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/20_dec_21/covariates_PCs_noMPV_bolt.csv --maxMissingPerSnp=

## (not used) PLINK with MPV

In [29]:
Y = X[["FID","IID"] + ["ADP_pred"]]
C = X[["FID","IID"] + ["AGE","cohort","SEX","MPV"] + ["PC%d" % d for d in range(1,10)]]

Y.to_csv("%s/%s/hippo_ADP.csv" % (export_folder,tag), index=False,header=True, sep="\t")
C.to_csv("%s/%s/covariates_PCs_MPV.csv" % (export_folder,tag), index=False,header=True, sep="\t")

In [30]:
n_splits = 12
for k in range(1,23):
    command = "/home/hv270/plink2 "

    command += "--bfile /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_%d " % k
    command += "--glm hide-covar"
    command += " --keep %s/king-cutoff.king.cutoff.in.id" % export_folder
    command += " --pheno %s/%s/hippo_ADP.csv" % (export_folder, tag)
    command += " --covar %s/%s/covariates_PCs_MPV.csv" % (export_folder, tag)
    command += " --out %s/%s/chr_%d_ADP_MPV" % (export_folder, tag,k)
    command += " --threads %d" % n_splits
    command += " --allow-no-sex"
    command += " --variance-standardize"
    
    f_name = "/home/hv270/interval_slurm_calls/call_chr_%d_sysmex.sh" % k
    f = open(f_name,"w+")
    f.write("#!/bin/sh\n")

    f.write("#SBATCH -J chr_%d\n" % k)
    f.write("#SBATCH -A ASTLE-SL2-CPU\n")
    f.write("#SBATCH -N 1\n")
    f.write("#SBATCH -n %d\n" % n_splits)
    #f.write("#SBATCH -c %d\n" % n_splits)
    f.write("#SBATCH --time=0:15:00\n")
    f.write("#SBATCH -p skylake-himem\n")
    f.write("#SBATCH --output=/home/hv270/slurm_log/%s_%s_ADP_MPV.out\n" % (tag,k))
    f.write("export OMP_NUM_THREADS=%d\n" % n_splits)

    f.write(". /etc/profile.d/modules.sh\n")
    f.write("module purge\n")
    f.write("module load rhel7/default-peta4\n")
    f.write(command)
    f.close()
    print(command)
    
    p = subprocess.Popen("sbatch %s" % f_name,shell=True)
    p.wait()

/home/hv270/plink2 --bfile /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_1 --glm hide-covar --keep /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/king-cutoff.king.cutoff.in.id --pheno /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/25_oct/hippo_ADP.csv --covar /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/25_oct/covariates_PCs_MPV.csv --out /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/25_oct/chr_1_ADP_MPV --threads 12 --allow-no-sex --variance-standardize
/home/hv270/plink2 --bfile /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_2 --glm hide-covar --keep /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/king-cutoff.king.cutoff.in.id --pheno /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/25_oc

/home/hv270/plink2 --bfile /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_14 --glm hide-covar --keep /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/king-cutoff.king.cutoff.in.id --pheno /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/25_oct/hippo_ADP.csv --covar /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/25_oct/covariates_PCs_MPV.csv --out /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/25_oct/chr_14_ADP_MPV --threads 12 --allow-no-sex --variance-standardize
/home/hv270/plink2 --bfile /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hv270/interval_genotypes/chr_15 --glm hide-covar --keep /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/king-cutoff.king.cutoff.in.id --pheno /home/hv270/rds/rds-who1000-cbrc/user/wja24/shared/hippolyte_only/interval_processing/25